## Dataset Audios creation

##### Randomi MIDI file creation

In [1]:
import random
from basic_pitch import ICASSP_2022_MODEL_PATH, inference

In [2]:
def create_individual(effects, effect_structure):
    n_effects_chosen = random.randint(1, len(effects))
    #n_effects_chosen = 1
    selected_effects = random.sample(effects, n_effects_chosen)
    
    individ = {}
    for effect in selected_effects:
        if effect in effect_structure:
            structure = effect_structure[effect]
            individ[effect] = {
                param: round(random.uniform(range_[0], range_[1]), 2) 
                for param, (_, range_) in structure.items()
            }
    return individ

In [3]:
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (1.0, 20.0)), },# Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)), },# Delay
    2: { "drive_db": ('float', (1.0, 20.0)), },# Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },# Gain
    4: { "depth": ('float', (0.2, 0.6)), },# Phaser
    5: { "wet_level": ('float', (0.2, 0.6)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [81]:
create_individual(effects, effect_structure)

{2: {'drive_db': 3.56}}

Testing similarity between 2 individuals

In [4]:
import random
import pretty_midi

def generate_random_instrument_midi(filename, program, min_duration, max_duration, min_note, max_note):
    # Create a PrettyMIDI object
    midi_data = pretty_midi.PrettyMIDI()
    
    # Create an Instrument instance for a distortion instrument
    instrument = pretty_midi.Instrument(program=program)
    
    # Determine the length of the MIDI in seconds
    duration = random.randint(min_duration, max_duration)
    min_note_number = pretty_midi.note_name_to_number(min_note)
    max_note_number = pretty_midi.note_name_to_number(max_note)
    
    # Generate random notes
    current_time = 0.0
    while current_time < duration:
        note_number = random.randint(min_note_number, max_note_number)
        
        # Random note duration between 0.1 and 1.0 seconds
        note_duration = random.uniform(0.1, 2.0)
        
        # Ensure that the note ends before the total duration
        note_end_time = min(current_time + note_duration, duration)
        
        # Create a Note instance and add it to the instrument instrument
        note = pretty_midi.Note(
            velocity=random.randint(60, 127),  # Random velocity
            pitch=note_number,
            start=current_time,
            end=note_end_time
        )
        instrument.notes.append(note)
        
        # Update the current time
        current_time = note_end_time
    
    # Add the instrument instrument to the PrettyMIDI object
    midi_data.instruments.append(instrument)
    
    # Write out the MIDI data to a file
    midi_data.write(filename)
    
    return midi_data


In [81]:
midi_file = 'output.mid'
program = 30
min_note = 'E2'
max_note = 'E6' # Generate a random note between MIDI note 40 (E2) and 88 (E6) so basically all notes of the guitar
min_duration = 5
max_duration = 15

midi_data = generate_random_instrument_midi(midi_file, program, min_duration, max_duration, min_note, max_note)

# Using dir() to list all properties and methods
print(dir(midi_data))

# Using __dict__ to see instance attributes
print(midi_data.__dict__)

# Accessing some specific properties
print("Tempo changes:", midi_data.get_tempo_changes())
print("Time signature changes:", midi_data.time_signature_changes)
print("Instruments:", midi_data.instruments[0].notes)
print(midi_data)


['_PrettyMIDI__tick_to_time', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_load_instruments', '_load_metadata', '_load_tempo_changes', '_tick_scales', '_update_tick_to_time', 'adjust_times', 'estimate_beat_start', 'estimate_tempi', 'estimate_tempo', 'fluidsynth', 'get_beats', 'get_chroma', 'get_downbeats', 'get_end_time', 'get_onsets', 'get_piano_roll', 'get_pitch_class_histogram', 'get_pitch_class_transition_matrix', 'get_tempo_changes', 'instruments', 'key_signature_changes', 'lyrics', 'remove_invalid_notes', 'resolution', 'synthesize', 'text_events', 'tick_to_time', 'time_signature_changes', 'time_to_tick', 'write']
{'resolution': 220, '_tick_scales': [(0, 0.0022727272727

In [5]:
from pedalboard import Pedalboard, Reverb, Chorus, Distortion, Delay, Phaser, Compressor, Gain, Clipping
from pedalboard.io import AudioFile

def create_effected_audio(board, file_path, output_file):
    with AudioFile(file_path) as f:
        with AudioFile(output_file, 'w', f.samplerate, f.num_channels) as o:
            while f.tell() < f.frames:
                chunk = f.read(f.samplerate)
                effected = board(chunk, f.samplerate, reset=False)
                o.write(effected)

In [6]:
from pedalboard.io import AudioFile

def create_audio(file_path, output_file):
    with AudioFile(file_path) as f:
        with AudioFile(output_file, 'w', f.samplerate, f.num_channels) as o:
            while f.tell() < f.frames:
                chunk = f.read(f.samplerate)
                o.write(chunk)

##### MP3 creation of the MIDI file

In [7]:
from midi2audio import FluidSynth 
from pydub import AudioSegment

def midi_to_mp3(midi_file, mp3_file, soundfont):
    #convert MIDI to WAV using FluidSynth
    fs = FluidSynth(soundfont)
    wav_file = midi_file.replace('.midi', '.wav').replace('.mid', '.wav')
    fs.midi_to_audio(midi_file, wav_file)

    #convert WAV to MP3 using pydub
    sound = AudioSegment.from_wav(wav_file)
    sound.export(mp3_file, format="mp3")

In [22]:
midi_file = "output.mid"
mp3_file = "output.mp3"
soundfont = '../audio2midi2audio/FluidR3_GM.sf2'  # Path to your SoundFont file

midi_to_mp3(midi_file, mp3_file, soundfont)

Conversion complete: output.mp3


##### From MIDI back to MP3

In [8]:
def mp3_to_midi(audio_path, midi_path):
    _, midi_data, __ = inference.predict(
        audio_path,    
        model_or_model_path = ICASSP_2022_MODEL_PATH, 
        onset_threshold = 0.5, #note segmentation 1) how easily a note should be split into two. (split notes <- ..0.5.. -> merge notes)
        frame_threshold = 0.3, #model confidence threshold 2) the model confidence required to create a note. (more notes <- ..0.3.. -> fewer notes)
    )

    for instrument in midi_data.instruments:
        instrument.program = 30 #distortion guitar program

    midi_data.write(midi_path)
    
    return midi_data

## DT creation

### DT creation for guitar songs

In [13]:
import pandas as pd

def audios_dataset_creation(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map):
    for i in range(n_data):
        generate_random_instrument_midi(midi_file, program, min_duration, max_duration, min_note, max_note)
        midi_to_mp3(midi_file, mp3_file, soundfont)
        individual = create_individual(effects, effect_structure)
        board = Pedalboard([])
        for effect_key, params in individual.items():
            effect_class = globals()[effects_map[effect_key]]
            board.append(effect_class(**params))
            
        output_file = str(i) + ".mp3"
        output_file_clean = str(i) + "_clean.mp3"
        create_effected_audio(board, mp3_file, "audios_low_ranges_w_clean/" + output_file)
        create_audio(mp3_file, "audios_low_ranges_w_clean/" + output_file_clean)
        data["audio_name"].append(output_file)
        data["applied_effects"].append(individual)
        data["GA_effects"].append("")
        data["similarity_indv"].append("")
    
    # Once all the data is collected, you can create the DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"CSV file '{csv_filename}' created successfully.")

### DT creation for violin songs

In [28]:
import pandas as pd

def audios_dataset_creation_violin(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration):
    for i in range(n_data):
        print(f'{i} audio created')
        generate_random_instrument_midi(midi_file, program, min_duration, max_duration, min_note, max_note)
        midi_to_mp3(midi_file, mp3_file, soundfont)
            
        output_file = str(i) + ".mp3"
        create_audio(mp3_file, "violin_audios/" + output_file)

        data["audio_name"].append(output_file)
        data["GA_effects"].append("")
    
    # Once all the data is collected, you can create the DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"CSV file '{csv_filename}' created successfully.")

Small test

In [29]:
individ= {2: {'drive_db': 8.06},
 1: {'delay_seconds': 2.52},
 7: {'threshold_db': -0.97},
 3: {'gain_db': -7.94}}
board = Pedalboard([])
for effect_key, params in individ.items():
    effect_class = globals()[effects_map[effect_key]]
    board.append(effect_class(**params))
            
print(board)
output_file = "G4_guitar_TEST.mp3"
create_effected_audio(board, '../audios/G4_guitar.mp3', output_file)

<Pedalboard with 4 plugins: [<pedalboard.Distortion drive_db=8.06 at 00000208D31CB0D0>, <pedalboard.Delay delay_seconds=2.52 feedback=0 mix=0.5 at 00000208D30B9AA0>, <pedalboard.Clipping threshold_db=-0.97 at 00000208D32AC400>, <pedalboard.Gain gain_db=-7.94 at 00000208D31B6DD0>]>


In [14]:
data = {
    "audio_name": [],
    "applied_effects": [], 
    "GA_effects": [],
    "similarity_indv": []
}

n_data = 250
midi_file = 'output.mid'
mp3_file = 'output.mp3'
csv_filename = "../results/dataset_audios_guitar_low_ranges_w_clean.csv"
soundfont = '../audio2midi2audio/FluidR3_GM.sf2'  # Path to your SoundFont file
program = 30
min_note = 'E2'
max_note = 'E6' # Generate a random note between MIDI note 40 (E2) and 88 (E6)
min_duration = 10
max_duration = 20
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (1.0, 20.0)), },# Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)), },# Delay
    2: { "drive_db": ('float', (1.0, 20.0)), },# Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },# Gain
    4: { "depth": ('float', (0.2, 0.6)), },# Phaser
    5: { "wet_level": ('float', (0.2, 0.6)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [15]:
audios_dataset_creation(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map)

CSV file '../results/dataset_audios_guitar_low_ranges_w_clean.csv' created successfully.


In [35]:
data = {
    "audio_name": [],
    "GA_effects": [], 
}

n_data = 500
midi_file = 'output.mid'
mp3_file = 'output.mp3'
csv_filename = "../results/dataset_audios_violin.csv"
soundfont = '../audio2midi2audio/FluidR3_GM.sf2'  # Path to your SoundFont file
program = 40 #violin
min_note = 'E2'
max_note = 'E6' # Generate a random note between MIDI note 40 (E2) and 88 (E6)
min_duration = 3
max_duration = 15

In [36]:
audios_dataset_creation_violin(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration)

0 audio created
1 audio created
2 audio created
3 audio created
4 audio created
5 audio created
6 audio created
7 audio created
8 audio created
9 audio created
10 audio created
11 audio created
12 audio created
13 audio created
14 audio created
15 audio created
16 audio created
17 audio created
18 audio created
19 audio created
20 audio created
21 audio created
22 audio created
23 audio created
24 audio created
25 audio created
26 audio created
27 audio created
28 audio created
29 audio created
30 audio created
31 audio created
32 audio created
33 audio created
34 audio created
35 audio created
36 audio created
37 audio created
38 audio created
39 audio created
40 audio created
41 audio created
42 audio created
43 audio created
44 audio created
45 audio created
46 audio created
47 audio created
48 audio created
49 audio created
50 audio created
51 audio created
52 audio created
53 audio created
54 audio created
55 audio created
56 audio created
57 audio created
58 audio created
59 audi

### Fill dataset with random effects as baseline

In [9]:
import pandas as pd

def fill_dataset_with_random_individuals(n_data, csv_filename, effects, effect_structure):
    df_random = pd.read_csv(csv_filename)

    random_effects = []
    for _ in range(n_data):
        individual2 = create_individual(effects, effect_structure)
        random_effects.append(individual2)
    
    df_random['random_effects'] = pd.Series(random_effects)
    
    df_random.to_csv(csv_filename, index=False)

    print(f"CSV file '{csv_filename}' created successfully.")

In [10]:
n_data = 250
csv_filename = "../results/dataset_audios_guitar_low_ranges_w_clean_random.csv"
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (1.0, 20.0)), },# Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)), },# Delay
    2: { "drive_db": ('float', (1.0, 20.0)), },# Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },# Gain
    4: { "depth": ('float', (0.2, 0.6)), },# Phaser
    5: { "wet_level": ('float', (0.2, 0.6)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

fill_dataset_with_random_individuals(n_data, csv_filename, effects, effect_structure)

CSV file '../results/dataset_audios_guitar_low_ranges_w_clean_random.csv' created successfully.
